# ToDo:
- preprocess text - lemmatization, remove stop words etc. 
- utlise preprocessed text
- Try DistilBert https://www.kaggle.com/code/alexia/kerasnlp-starter-notebook-disaster-tweets
- sklearn pipelines
- cross validation


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import f1_score, r2_score, accuracy_score
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import xgboost
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

## Preproc

In [ ]:
def lemmatize_remove_stop(doc):
    return (" ".join([token.lemma_ for token in nlp(doc) if not token.is_stop])).strip().lower()

In [ ]:
train_df['text_preproc'] = train_df['text'].apply(lemmatize_remove_stop)
test_df['text_preproc'] = test_df['text'].apply(lemmatize_remove_stop)

In [ ]:
X = train_df.set_index('id')
y = train_df.set_index('id')['target']

X_test = test_df.set_index('id')

In [ ]:
count_vectorizer = feature_extraction.text.CountVectorizer(max_features=5000)
X_bow_raw = count_vectorizer.fit_transform(X['text'])
X_test_bow_raw = count_vectorizer.transform(X_test['text'])

count_vectorizer2 = feature_extraction.text.CountVectorizer(max_features=5000)
X_bow = count_vectorizer2.fit_transform(X['text_preproc']);
X_test_bow = count_vectorizer2.transform(X_test['text_preproc']);

In [ ]:
tfidf_vectorizer = feature_extraction.text.TfidfVectorizer(max_features=5000)
X_tfidf_raw = tfidf_vectorizer.fit_transform(X['text'])
X_test_tfidf_raw = tfidf_vectorizer.transform(X_test['text'])

tfidf_vectorizer2 = feature_extraction.text.TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer2.fit_transform(X['text_preproc']);
X_test_tfidf = tfidf_vectorizer2.transform(X_test['text_preproc']);

In [ ]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# # Bag of words

# count_vectorizer = feature_extraction.text.CountVectorizer(max_features=5000)
# X_bow = count_vectorizer.fit_transform(X['text']);

# X_train_bow = count_vectorizer.transform(X_train['text'])
# X_test_bow = count_vectorizer.transform(X_test['text'])

# # TF-IDF

# tf_idf_trans = feature_extraction.text.TfidfVectorizer(max_features=5000)
# X_tf_idf = tf_idf_trans.fit_transform(X['text'])

# X_train_tf_idf = tf_idf_trans.transform(X_train)
# X_test_tf_idf = tf_idf_trans.transform(X_test)

## Modelling

In [ ]:
pd.DataFrame(cross_validate(MultinomialNB(), X_tfidf, y, scoring=["f1", "accuracy", "r2"], return_train_score=True)).agg(['mean', 'std'])

In [ ]:
def fit_return_metrics(X_train_input, y_train_input, X_test_input,y_test_input, model):
    """Fit the model, and return train and test metrics"""

    model.fit(X_train_input, y_train_input);
    y_train_pred = model.predict(X_train_input)
    y_test_pred = model.predict(X_test_input)

    # train
    
    metric_set = dict(f1=f1_score, 
                      r2=r2_score,
                      accuracy = accuracy_score)
    
    def metrics_dict(y_act, y_pred):
        return{metric : metric_func(y_act, y_pred) for metric, metric_func in metric_set.items()}
      
    metrics_list = [dict(regime="train", **metrics_dict(y_train_input,y_train_pred)),
                   dict(regime="test", **metrics_dict(y_test_input, y_test_pred))
                   ]
    
    return pd.DataFrame(metrics_list).set_index("regime").unstack()

In [ ]:
pd.DataFrame([fit_return_metrics(X_train_bow, y_train, X_test_bow,y_test,xgboost.XGBClassifier()).rename('xgboost_bow'),
              fit_return_metrics(X_train_tf_idf, y_train, X_test_tf_idf,y_test,xgboost.XGBClassifier()).rename('xgboost_tfidf'),
#               fit_return_metrics(X_train_bow.toarray(), y_train, X_test_bow.toarray(),y_test,GaussianProcessClassifier()).rename('gaussianprocess_bow'),
#               fit_return_metrics(X_train_bow.toarray(), y_train, X_test_tf_idf.toarray(),y_test,GaussianProcessClassifier()).rename('gaussianprocess_tfidf'),
#               fit_return_metrics(X_train_bow.toarray(), y_train, X_test_bow.toarray(),y_test,MultinomialNB()).rename('naivebayes_bow'),
#               fit_return_metrics(X_train_bow.toarray(), y_train, X_test_tf_idf.toarray(),y_test,MultinomialNB()).rename('naivebayes_tfidf')
             ])
 
 

In [ ]:
# see this notebook for BERT implementation 
# https://www.kaggle.com/code/alexia/kerasnlp-starter-notebook-disaster-tweets

In [ ]:
best_model = xgboost.XGBClassifier()

best_model.fit(X_bow, y)

X_test_bow = count_vectorizer.transform(test_df['text'])

y_pred_test  = best_model.predict(X_test_bow)

In [ ]:
# Create a submission dataframe
submission = pd.DataFrame({'id': test_df['id'], 'target': y_pred_test})

# Write the submission dataframe to a CSV file
submission.to_csv('submission.csv', index=False)

submission.head()